In [1]:
'''
   Name: Seema Goel
   Date : 22 April 2018
   version 1.1.0
   Purpose: To load the videos,extract suitable features and train a Machine Learning model to classify each video as walking or jogging.

'''


import os
import sys
import argparse
import csv
import numpy as np
import imageio
imageio.plugins.ffmpeg.download()
from random import randint
from PIL import Image

In [2]:
'''
    identify_classes  is the function to figure out lables in the dataset
    VARIABLES:
     path : path to dataset folder
     classes : list of classes to be classified
     l_folders: classes folder in train dataset
     labels : are classes/lables to be identifed
     below function is identifing lables/classes 
'''
def identify_classes(path):
    # empty list of lables
    classes = []
    
    l_folders  = os.listdir(path)
    index = 0
    
    # looping over Directory structure to extract folders name as classes
    for labels in sorted(l_folders):
        label_folder_path = os.path.join(path, labels)
        if os.path.isdir(label_folder_path):
            group_folders = os.listdir(label_folder_path)
            for group_folder in group_folders:
                # no annototions required here 
                if group_folder != 'Annotation':
                    classes.append([os.path.join(label_folder_path, group_folder), index])
                    index += 1

    return classes

In [3]:
'''
  video_Files_split is splitiing the dataset into training and test with 80% and 20%
 Variables:
 group_count : number of classes/lables
  indices: holds evenly spaced values
  train_count: hold number of files in training set
  test_count: number of files in test set
  test, train: list to hold name of file in both sets
  video_files = list containing video files
'''

def video_Files_split(imgroot, cats, pot,file_ext):
    train = []
    test = []
    
    for cat in cats:
        category_in_number = cats.index(cat)
        files = os.listdir(os.path.join(imgroot, cat))
        split = int(pot*len(files))
        for filename in (files[:split]):
            file_path =  os.path.join(imgroot,cat, filename)
            if os.path.isfile(file_path):
                file_path = os.path.abspath(file_path)
                ext = os.path.splitext(file_path)[1]
                
                if (ext == file_ext ):
                    train.append([filename,category_in_number])
        
        for filename in (files[split:]):
            file_path =  os.path.join(imgroot,cat, filename)
            if os.path.isfile(file_path):
                file_path = os.path.abspath(file_path)
                ext = os.path.splitext(file_path)[1]
                
                if (ext == file_ext ):
                    test.append([filename, category_in_number]) 
               
    return train, test
    
    

In [4]:
'''
    Path file names are written in CSV files
    
'''

def create_csv(items, file_path):
   
    if sys.version_info[0] < 3:
        with open(file_path, 'wb') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for item in items:
                writer.writerow(item)
    else:
        with open(file_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for item in items:
                writer.writerow(item)
    print("thumbsup")

In [7]:
cats = ['walking','jogging']
pot = 0.8
train,test = video_Files_split('data/train', cats, pot,'.avi')

In [8]:
len(train)

82

In [9]:
len(test)

22

In [10]:
create_csv(train, os.path.join('data/', 'train_map.csv'))
create_csv(test, os.path.join('data/', 'test_map.csv'))

thumbsup
thumbsup


#####  Reading video from path in csv, extract frames

##### Convert frames to numpy arrays and save in file


In [11]:
def select_features(video_file, sequence_length, is_training,width,height):
        '''
        Select a sequence of frames from video_file and return them as
        a Tensor.
        '''
        try:
            video_reader = imageio.get_reader(video_file, 'ffmpeg')
            num_frames   = len(video_reader)
            if sequence_length > num_frames:
                raise ValueError('Sequence length {} is larger then the total number of frames {} in {}.'.format(sequence_length, num_frames, video_file))

            # select which sequence frames to use.
            step = 1
            expanded_sequence = sequence_length
            if num_frames > 2*sequence_length:
                step = 2
                expanded_sequence = 2*sequence_length

            seq_start = int(num_frames/2) - int(expanded_sequence/2)
            if is_training:
                seq_start = randint(0, num_frames - expanded_sequence)

            frame_range = [seq_start + step*i for i in range(sequence_length)]            
            video_frames = []
            for frame_index in frame_range:
                video_frames.append(read_frame(video_reader.get_data(frame_index),width,height))
        
            return np.stack(video_frames, axis=1)
        except:
            print("Error reading video")

In [12]:
 '''
        Using best practices/augumentation of 3D CNN resizing the image to 128x171 first, then selecting a 112x112 crop.
 '''

def read_frame(data,width,height):
       
        try:
            if (width >= 171) or (height >= 128):
                raise ValueError("Target width need to be less than 171 and target height need to be less than 128.")
        
            image = Image.fromarray(data)
            image.thumbnail((171, 128), Image.ANTIALIAS)
        
            center_w = image.size[0] / 2
            center_h = image.size[1] / 2

            image = image.crop((center_w - width  / 2,
                                center_h - height / 2,
                                center_w + width  / 2,
                                center_h + height / 2))
        
            norm_image = np.array(image, dtype=np.float32)
            norm_image -= 127.5
            norm_image /= 127.5

            # (channel, height, width)
            return np.ascontiguousarray(np.transpose(norm_image, (2, 0, 1)))
        except:
            print("Error reading frame")

In [14]:
def read_video(map_file, label_count, is_training, limit_epoch_size=sys.maxsize):
        '''
        Load video file paths and their corresponding labels.
        '''
        width           = 112
        height          = 112
        sequence_length = 16
        channel_count   = 3
        is_training     = is_training
        video_files     = []
        targets         = []
        batch_start     = 0

        try:
            map_file_dir = os.path.dirname(map_file)

            with open(map_file) as csv_file:
                data = csv.reader(csv_file)
                for row in data:
                    if (row[1] == '0'):
                        video_files.append(os.path.join(map_file_dir,'train/walking',row[0]))
                    elif (row[1] == '1'):
                        video_files.append(os.path.join(map_file_dir,'train/jogging',row[0]))
                   
                    target = [0.0] * label_count
                    target[int(row[1])] = 1.0
                    targets.append(target)

            indices = np.arange(len(video_files))
            if is_training:
                np.random.shuffle(indices)
            epoch_size = min(len(video_files), limit_epoch_size)
            vectored_video_files = []
            for video_file in video_files:
                vectored_video_file = select_features(video_file, sequence_length, is_training,width,height)
                print(vectored_video_file)
                vectored_video_files.append(vectored_video_file)
            
            return vectored_video_files,targets
        except:
            print("error converting to vector form")

In [15]:
num_output_classes = 2
train_reader = read_video(os.path.join('data/', 'train_map.csv'), num_output_classes, True)
test_reader  = read_video(os.path.join('data/', 'test_map.csv'), num_output_classes, True)

[[[[ 0.8666667   0.8666667   0.8666667  ...  0.9607843   0.9607843
     0.9607843 ]
   [ 0.8980392   0.8901961   0.8901961  ...  0.9607843   0.9607843
     0.9607843 ]
   [ 0.8901961   0.8901961   0.8980392  ...  0.92941177  0.92941177
     0.94509804]
   ...
   [ 0.7411765   0.7490196   0.70980394 ...  0.7254902   0.6862745
     0.7254902 ]
   [ 0.79607844  0.7882353   0.75686276 ...  0.7411765   0.7882353
     0.7882353 ]
   [ 0.67058825  0.6         0.75686276 ...  0.654902    0.5921569
     0.6313726 ]]

  [[ 0.8745098   0.8745098   0.8745098  ...  0.9529412   0.9529412
     0.9529412 ]
   [ 0.8901961   0.8745098   0.8901961  ...  0.9607843   0.9607843
     0.9607843 ]
   [ 0.8901961   0.8901961   0.8901961  ...  0.92941177  0.94509804
     0.92941177]
   ...
   [ 0.7411765   0.79607844  0.6862745  ...  0.7254902   0.7254902
     0.7882353 ]
   [ 0.73333335  0.85882354  0.7882353  ...  0.7490196   0.8352941
     0.69411767]
   [ 0.7254902   0.58431375  0.7411765  ...  0.75686276  0

[[[[-0.37254903 -0.3647059  -0.3647059  ... -0.20784314 -0.20784314
    -0.2       ]
   [-0.34901962 -0.33333334 -0.33333334 ... -0.20784314 -0.21568628
    -0.23137255]
   [-0.35686275 -0.34901962 -0.34901962 ... -0.21568628 -0.21568628
    -0.21568628]
   ...
   [-0.42745098 -0.3882353  -0.3254902  ... -0.20784314 -0.19215687
    -0.23137255]
   [-0.54509807 -0.49803922 -0.4745098  ... -0.23137255 -0.23137255
    -0.2       ]
   [-0.54509807 -0.54509807 -0.44313726 ... -0.19215687 -0.19215687
    -0.2       ]]

  [[-0.37254903 -0.3647059  -0.3647059  ... -0.20784314 -0.20784314
    -0.21568628]
   [-0.35686275 -0.34901962 -0.34901962 ... -0.23921569 -0.23137255
    -0.23921569]
   [-0.35686275 -0.34901962 -0.34901962 ... -0.21568628 -0.21568628
    -0.23137255]
   ...
   [-0.3882353  -0.40392157 -0.38039216 ... -0.23137255 -0.23921569
    -0.24705882]
   [-0.54509807 -0.49803922 -0.48235294 ... -0.23137255 -0.20784314
    -0.21568628]
   [-0.5294118  -0.5137255  -0.45882353 ... -0.20

[[[[0.79607844 0.79607844 0.8039216  ... 0.8666667  0.8901961
    0.8666667 ]
   [0.7882353  0.8039216  0.79607844 ... 0.84313726 0.8745098
    0.8745098 ]
   [0.8039216  0.79607844 0.7647059  ... 0.85882354 0.8980392
    0.85882354]
   ...
   [0.7490196  0.6862745  0.7254902  ... 0.7411765  0.69411767
    0.84313726]
   [0.84313726 0.79607844 0.8509804  ... 0.73333335 0.70980394
    0.8039216 ]
   [0.7647059  0.8509804  0.8117647  ... 0.7254902  0.79607844
    0.75686276]]

  [[0.7882353  0.8117647  0.8039216  ... 0.8666667  0.8980392
    0.8666667 ]
   [0.7882353  0.8039216  0.8039216  ... 0.8509804  0.8745098
    0.8666667 ]
   [0.8117647  0.79607844 0.78039217 ... 0.8745098  0.90588236
    0.85882354]
   ...
   [0.7490196  0.6784314  0.7254902  ... 0.70980394 0.6862745
    0.8352941 ]
   [0.8352941  0.79607844 0.8352941  ... 0.7411765  0.73333335
    0.8117647 ]
   [0.79607844 0.84313726 0.8039216  ... 0.70980394 0.8039216
    0.7647059 ]]

  [[0.8039216  0.7882353  0.79607844 ... 

[[[[ 0.20784314  0.2627451   0.23921569 ...  0.41960785  0.34901962
     0.34901962]
   [ 0.31764707  0.29411766  0.2627451  ...  0.34901962  0.35686275
     0.35686275]
   [ 0.41960785  0.33333334  0.41960785 ...  0.31764707  0.33333334
     0.30980393]
   ...
   [ 0.09803922  0.06666667  0.06666667 ... -0.01176471  0.08235294
     0.09803922]
   [ 0.09019608  0.07450981  0.06666667 ...  0.04313726  0.01960784
     0.06666667]
   [ 0.02745098  0.07450981  0.02745098 ...  0.06666667  0.08235294
     0.06666667]]

  [[ 0.22352941  0.24705882  0.20784314 ...  0.4117647   0.3647059
     0.3647059 ]
   [ 0.3019608   0.29411766  0.25490198 ...  0.34901962  0.38039216
     0.37254903]
   [ 0.43529412  0.33333334  0.4117647  ...  0.34117648  0.31764707
     0.3019608 ]
   ...
   [ 0.11372549  0.06666667  0.07450981 ...  0.00392157  0.06666667
     0.09803922]
   [ 0.07450981  0.07450981  0.04313726 ...  0.03529412  0.00392157
     0.04313726]
   [ 0.01960784  0.05882353  0.03529412 ...  0.090

[[[[ 0.09803922  0.12156863  0.11372549 ...  0.2         0.23137255
     0.23137255]
   [ 0.12941177  0.12941177  0.12941177 ...  0.20784314  0.2
     0.20784314]
   [ 0.14509805  0.19215687  0.16862746 ...  0.16862746  0.16862746
     0.18431373]
   ...
   [ 0.05882353  0.08235294  0.03529412 ...  0.00392157  0.02745098
     0.09019608]
   [-0.01960784  0.13725491  0.11372549 ...  0.04313726 -0.02745098
     0.19215687]
   [ 0.03529412  0.01960784  0.01176471 ...  0.05882353  0.09019608
    -0.04313726]]

  [[ 0.09803922  0.11372549  0.09803922 ...  0.19215687  0.22352941
     0.23137255]
   [ 0.12941177  0.12156863  0.12941177 ...  0.19215687  0.18431373
     0.18431373]
   [ 0.13725491  0.18431373  0.16862746 ...  0.13725491  0.15294118
     0.2       ]
   ...
   [ 0.04313726  0.06666667  0.08235294 ... -0.02745098  0.00392157
     0.09019608]
   [-0.02745098  0.11372549  0.04313726 ... -0.01176471  0.03529412
     0.18431373]
   [ 0.07450981  0.09803922  0.01960784 ...  0.09803922 

...

[[[[ 0.12156863  0.1764706   0.14509805 ...  0.14509805  0.1764706
     0.15294118]
   [ 0.13725491  0.09019608  0.11372549 ...  0.13725491  0.19215687
     0.12156863]
   [ 0.11372549  0.11372549  0.09803922 ...  0.07450981  0.12156863
     0.07450981]
   ...
   [-0.05098039 -0.13725491 -0.01176471 ... -0.05098039 -0.01176471
    -0.09803922]
   [ 0.03529412  0.01960784  0.00392157 ...  0.15294118  0.09019608
     0.06666667]
   [-0.02745098  0.01176471 -0.08235294 ... -0.02745098 -0.07450981
    -0.06666667]]

  [[ 0.13725491  0.15294118  0.14509805 ...  0.12941177  0.15294118
     0.14509805]
   [ 0.12941177  0.12156863  0.11372549 ...  0.13725491  0.18431373
     0.11372549]
   [ 0.13725491  0.11372549  0.12156863 ...  0.09019608  0.13725491
     0.09019608]
   ...
   [-0.07450981 -0.19215687 -0.01960784 ... -0.08235294  0.01960784
    -0.08235294]
   [ 0.03529412  0.02745098  0.01176471 ...  0.16862746  0.11372549
     0.06666667]
   [-0.02745098  0.00392157 -0.09019608 ... -0.019

[[[[ 0.34117648  0.15294118  0.23921569 ...  0.08235294  0.12941177
     0.14509805]
   [ 0.2         0.23137255  0.16862746 ...  0.15294118  0.18431373
     0.2       ]
   [ 0.13725491  0.14509805  0.19215687 ...  0.08235294  0.09019608
     0.09803922]
   ...
   [ 0.00392157  0.08235294  0.14509805 ... -0.06666667 -0.16078432
    -0.01960784]
   [ 0.03529412  0.01960784  0.01960784 ... -0.04313726 -0.16078432
    -0.04313726]
   [ 0.04313726 -0.06666667 -0.02745098 ...  0.04313726 -0.05098039
     0.04313726]]

  [[ 0.3019608   0.15294118  0.2627451  ...  0.06666667  0.04313726
     0.09803922]
   [ 0.25490198  0.2         0.11372549 ...  0.15294118  0.14509805
     0.20784314]
   [ 0.11372549  0.15294118  0.12941177 ...  0.13725491  0.13725491
     0.05882353]
   ...
   [-0.03529412  0.09019608  0.13725491 ... -0.08235294 -0.09803922
     0.00392157]
   [ 0.01176471  0.01960784  0.01176471 ...  0.01960784 -0.15294118
    -0.01960784]
   [-0.09019608 -0.07450981  0.06666667 ...  0.02

[[[[ 0.14509805  0.15294118  0.12941177 ...  0.1764706   0.04313726
     0.22352941]
   [ 0.19215687  0.04313726  0.12941177 ...  0.07450981  0.14509805
     0.14509805]
   [ 0.07450981  0.11372549  0.13725491 ...  0.15294118  0.07450981
     0.09803922]
   ...
   [ 0.11372549  0.33333334  0.01960784 ...  0.01176471  0.06666667
    -0.19215687]
   [ 0.14509805  0.09019608  0.01960784 ... -0.01176471  0.05882353
    -0.12941177]
   [ 0.25490198 -0.09019608  0.02745098 ...  0.06666667  0.08235294
     0.01960784]]

  [[ 0.12156863  0.12941177  0.12941177 ...  0.19215687  0.01176471
     0.23137255]
   [ 0.16862746  0.09019608  0.14509805 ...  0.07450981  0.13725491
     0.1764706 ]
   [ 0.08235294  0.09019608  0.12156863 ...  0.12941177  0.09019608
     0.11372549]
   ...
   [ 0.03529412  0.13725491 -0.13725491 ... -0.04313726  0.07450981
    -0.2       ]
   [ 0.05882353 -0.03529412  0.02745098 ... -0.01176471  0.01176471
    -0.12941177]
   [ 0.2         0.02745098  0.07450981 ...  0.01

[[[[-0.34901962 -0.3647059  -0.34901962 ... -0.23921569 -0.25490198
    -0.2627451 ]
   [-0.3647059  -0.37254903 -0.34901962 ... -0.25490198 -0.27058825
    -0.27058825]
   [-0.3647059  -0.3647059  -0.3647059  ... -0.24705882 -0.25490198
    -0.27058825]
   ...
   [-0.49019608 -0.49803922 -0.49803922 ... -0.3019608  -0.28627452
    -0.3019608 ]
   [-0.5137255  -0.5137255  -0.5137255  ... -0.30980393 -0.29411766
    -0.3019608 ]
   [-0.5137255  -0.52156866 -0.5294118  ... -0.29411766 -0.29411766
    -0.2627451 ]]

  [[-0.3647059  -0.35686275 -0.3254902  ... -0.24705882 -0.24705882
    -0.25490198]
   [-0.3647059  -0.33333334 -0.33333334 ... -0.28627452 -0.29411766
    -0.27058825]
   [-0.3647059  -0.35686275 -0.3647059  ... -0.25490198 -0.25490198
    -0.28627452]
   ...
   [-0.52156866 -0.5137255  -0.52156866 ... -0.28627452 -0.27058825
    -0.3019608 ]
   [-0.52156866 -0.49803922 -0.5372549  ... -0.28627452 -0.27058825
    -0.29411766]
   [-0.5137255  -0.5294118  -0.52156866 ... -0.29

[[[[ 0.2627451   0.2627451   0.2784314  ...  0.33333334  0.33333334
     0.34901962]
   [ 0.20784314  0.20784314  0.20784314 ...  0.29411766  0.29411766
     0.3019608 ]
   [ 0.20784314  0.22352941  0.20784314 ...  0.24705882  0.25490198
     0.25490198]
   ...
   [ 0.13725491  0.15294118  0.13725491 ...  0.01960784  0.06666667
     0.00392157]
   [ 0.09019608  0.05882353  0.07450981 ... -0.01960784 -0.03529412
     0.01960784]
   [ 0.08235294  0.12156863  0.12941177 ... -0.05098039  0.03529412
     0.02745098]]

  [[ 0.25490198  0.24705882  0.24705882 ...  0.34117648  0.33333334
     0.34117648]
   [ 0.23137255  0.2         0.22352941 ...  0.2784314   0.29411766
     0.28627452]
   [ 0.19215687  0.22352941  0.2        ...  0.24705882  0.2627451
     0.2627451 ]
   ...
   [ 0.12941177  0.13725491  0.09803922 ... -0.02745098 -0.01960784
    -0.03529412]
   [ 0.11372549  0.09803922  0.08235294 ...  0.05882353 -0.01960784
     0.11372549]
   [ 0.06666667  0.08235294  0.11372549 ... -0.121

In [16]:
import numpy
numpy.save('data/train_data.npy', train_reader[0])

In [17]:
numpy.save('data/train_label.npy', train_reader[1])

In [18]:
numpy.save('data/test_data.npy', test_reader[0])

In [19]:
numpy.save('data/test_label.npy', test_reader[1])

In [20]:
data_npfile = np.load('data/test_label.npy')

In [21]:
len(train_reader[1])

82

In [22]:
len(test_reader[1])

22